In [1]:
import os
import sys
import pickle
from time import time
import pandas as pd

# Local Imports
path = str(os.path.expanduser('~')) + '/git/predictEPL/config'
sys.path.append(path)
import paths

sys.path.append(paths.UTILS)
import emolex
import useful_methods

In [2]:
# *******************************************************
# *******************************************************

# Game Infos
os.chdir(paths.READ_PATH_GAME_INFO)
dfGameInfos = useful_methods.csv_dic_df('game_infos.csv')

# Convert number strings to integers
dfGameInfos['GW'] = [int(GW) for GW in dfGameInfos['GW']]
dfGameInfos['score_ht_home'] = [int(number) for number in dfGameInfos['score_ht_home']]
dfGameInfos['score_ht_away'] = [int(number) for number in dfGameInfos['score_ht_away']]
dfGameInfos['score_ft_home'] = [int(number) for number in dfGameInfos['score_ft_home']]
dfGameInfos['score_ft_away'] = [int(number) for number in dfGameInfos['score_ft_away']]

# 'win': 1, 'lose': 0, 'draw': 2
def Labeling(goal_diff):
    if goal_diff > 0:
        return 3
    elif goal_diff < 0:
        return 0
    else:
        return 1
dfGameInfos['goal_diff_home'] = dfGameInfos.score_ft_home - dfGameInfos.score_ft_away
dfGameInfos['goal_diff_away'] = dfGameInfos.score_ft_away - dfGameInfos.score_ft_home
dfGameInfos['result_home'] = [Labeling(goal_diff) for goal_diff in dfGameInfos.goal_diff_home]
dfGameInfos['result_away'] = [Labeling(goal_diff) for goal_diff in dfGameInfos.goal_diff_away]


In [3]:
gws = list(set(dfGameInfos.GW))
teams = list(set(dfGameInfos.home_team))
teams.sort()

dfTeamWLD = pd.DataFrame()
dfTeamWLD['team'] = teams

for gw in gws:
    
    df = dfGameInfos[dfGameInfos.GW == gw].copy().reset_index(drop=True)
    dic_res = {}

    for i in range(10):
        home_team = df.loc[i]['home_team']
        away_team = df.loc[i]['away_team']
        result_home = df.loc[i]['result_home']
        result_away = df.loc[i]['result_away']

        dic_res[home_team] = result_home
        dic_res[away_team] = result_away

    team_wlds = sorted(dic_res.items())
    wlds = [team_wld[1] for team_wld in team_wlds]

    dfTeamWLD[gw] = wlds

dfTeamWLD

,team,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,Arsenal,0,3,1,3,3,0,3,3,3,...,3,3,0,3,3,1,1,0,1,3
1,Bournemouth,0,0,3,1,0,3,0,1,0,...,3,3,1,0,1,0,3,1,3,0
2,Chelsea,1,0,3,0,0,3,1,0,3,...,0,3,1,1,3,1,1,3,1,1
3,City,3,3,3,3,3,0,0,3,3,...,3,0,3,1,3,1,3,1,3,0
4,Crystal,3,0,3,3,0,0,3,3,0,...,3,3,1,1,0,0,0,0,0,1
5,Everton,1,3,0,1,3,1,3,1,0,...,1,0,3,0,1,1,1,0,3,3
6,Leicester,3,3,1,1,3,1,0,3,1,...,3,3,0,1,1,3,1,3,3,3
7,Liverpool,3,3,1,0,0,1,3,1,1,...,1,0,3,3,0,1,0,3,0,1
8,Newcastle,1,0,1,0,0,0,1,0,3,...,3,1,0,0,0,1,3,0,0,3
9,Norwich,0,3,1,0,3,1,1,0,0,...,1,3,0,3,3,0,0,0,0,0


In [4]:
# Save as CSV
useful_methods.DFtoCSV(dfTeamWLD, paths.DATA_HOME + "EPL/", 'info_team_wld', index=False)
print("[Saved in]: %s" % (paths.DATA_HOME + "EPL/" + 'info_team_wld.csv'))

[Saved in]: /Users/Bya/Dropbox/Research/datas/EPL/info_team_wld.csv


In [5]:
dfBe4GameSum = pd.DataFrame(columns=dfTeamWLD.columns)
dfBe4GameSum['team'] = dfTeamWLD['team']

for gw in gws:
    try:
        dfBe4GameSum[gw] = dfTeamWLD[gw-4] + dfTeamWLD[gw-3] + dfTeamWLD[gw-2] + dfTeamWLD[gw-1]
    except:
        continue

dfBe4GameSum = dfBe4GameSum.fillna(0).copy()
dfBe4GameSum

,team,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,Arsenal,0,0,0,0,7,10,7,9,9,...,7,5,5,7,10,9,9,9,7,8
1,Bournemouth,0,0,0,0,4,4,7,4,4,...,1,2,5,8,10,10,7,5,2,4
2,Chelsea,0,0,0,0,4,3,6,4,4,...,3,4,4,4,4,4,5,8,6,6
3,City,0,0,0,0,12,12,9,6,6,...,5,7,4,6,6,6,7,7,8,8
4,Crystal,0,0,0,0,9,6,6,6,6,...,4,7,7,7,10,8,8,5,2,1
5,Everton,0,0,0,0,5,7,5,8,8,...,7,8,6,6,3,5,4,4,5,3
6,Leicester,0,0,0,0,8,8,6,5,7,...,12,10,10,10,10,9,7,5,5,6
7,Liverpool,0,0,0,0,7,4,2,4,5,...,7,9,6,7,4,4,7,6,7,4
8,Newcastle,0,0,0,0,2,1,1,1,1,...,4,4,6,6,7,7,4,1,1,4
9,Norwich,0,0,0,0,4,7,5,5,5,...,3,4,4,2,5,4,7,9,6,6


In [6]:
# Save as CSV
useful_methods.DFtoCSV(dfBe4GameSum, paths.DATA_HOME + "EPL/", 'info_before_4game_sum', index=False)
print("[Saved in]: %s" % (paths.DATA_HOME + "EPL/" + 'info_before_4game_sum.csv'))

[Saved in]: /Users/Bya/Dropbox/Research/datas/EPL/info_before_4game_sum.csv


In [9]:
df = useful_methods.csv_dic_df(paths.DATA_HOME + "EPL/all_game_hash_review_pn_score.csv")
dfBe4GameSum = useful_methods.csv_dic_df(paths.DATA_HOME + "EPL/info_before_4game_sum.csv")

team_homes = list(df.home_team)
team_aways = list(df.away_team)
GWs = [int(gw) for gw in df.GW]

be_4game_sum_home = []
be_4game_sum_away = []

for index in range(len(GWs)):
    team_home = team_homes[index]
    team_away = team_aways[index]
    gw = GWs[index]

    be_4game_sum_home.append(int(dfBe4GameSum[dfBe4GameSum.team == team_home][str(gw)]))
    be_4game_sum_away.append(int(dfBe4GameSum[dfBe4GameSum.team == team_away][str(gw)]))

df['be_4game_sum_home'] = be_4game_sum_home
df['be_4game_sum_away'] = be_4game_sum_away
df

,GW,away_team,date,home_team,pn_away_neg,pn_away_pos,pn_home_neg,pn_home_pos,score_ft_away,score_ft_home,score_ht_away,score_ht_home,time,be_4game_sum_home,be_4game_sum_away
0,1,Tottenham,Sat. 8 Aug.,United,-1.0,-1.0,-1.0,-1.0,0,1,0,1,11:45,0,0
1,1,Watford,Sat. 8 Aug.,Everton,-1.0,-1.0,-1.0,-1.0,2,2,1,0,14:00,0,0
2,1,Sunderland,Sat. 8 Aug.,Leicester,-1.0,-1.0,-1.0,-1.0,2,4,0,3,14:00,0,0
3,1,Crystal,Sat. 8 Aug.,Norwich,-1.0,-1.0,-1.0,-1.0,3,1,1,0,14:00,0,0
4,1,Villa,Sat. 8 Aug.,Bournemouth,-1.0,-1.0,-1.0,-1.0,1,0,0,0,14:00,0,0
5,1,Swansea,Sat. 8 Aug.,Chelsea,-1.0,-1.0,-1.0,-1.0,2,2,1,2,16:30,0,0
6,1,WestHam,Sun. 9 Aug.,Arsenal,-1.0,-1.0,-1.0,-1.0,2,0,1,0,12:30,0,0
7,1,Southampton,Sun. 9 Aug.,Newcastle,-1.0,-1.0,-1.0,-1.0,2,2,1,1,12:30,0,0
8,1,Liverpool,Sun. 9 Aug.,Stoke,-1.0,-1.0,-1.0,-1.0,1,0,0,0,15:00,0,0
9,1,City,Mon. 10 Aug.,WestBromwich,-1.0,-1.0,-1.0,-1.0,3,0,2,0,19:00,0,0


In [10]:
team_homes = list(dfGameInfos.home_team)
team_aways = list(dfGameInfos.away_team)
GWs = list(dfGameInfos.GW)

be_4game_sum_home = []
be_4game_sum_away = []

for index in range(len(GWs)):
    team_home = team_homes[index]
    team_away = team_aways[index]
    gw = GWs[index]

    be_4game_sum_home.append(int(dfBe4GameSum[dfBe4GameSum.team == team_home][gw]))
    be_4game_sum_away.append(int(dfBe4GameSum[dfBe4GameSum.team == team_away][gw]))

dfGameInfos['be_4game_sum_home'] = be_4game_sum_home
dfGameInfos['be_4game_sum_away'] = be_4game_sum_away
dfGameInfos

KeyError: 1